In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [0]:
engagement_df = pd.read_csv('takehome_user_engagement.csv')
users_df = pd.read_csv("takehome_users.csv", encoding = "ISO-8859-1")

In [3]:
users_df.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [0]:
encoded_source = users_df['creation_source'].str.get_dummies()

In [0]:
users_df = users_df.join(encoded_source)

In [6]:
len(engagement_df['user_id'].unique())

8823

In [0]:
engagement_df['week'] = (pd.to_datetime(engagement_df['time_stamp'])
       .dt.floor('7d'))

In [0]:
temp = engagement_df.groupby(['user_id', 'week']).agg({'visited': np.sum})
temp[(temp['visited'] == 2) | (temp['visited'] == 1)] = 0

In [0]:
active_users = temp[(temp['visited'] > 2) | (temp['visited'] == 0)].groupby('user_id').agg({'visited': np.sum})
active_users = active_users.reset_index()

In [0]:
target = users_df.set_index('object_id').join(active_users.set_index('user_id'))
target['adopted_user'] = target['visited'] > 1
target = target.drop(['visited','creation_time','name','email','creation_source'], axis= 1)

In [11]:
target.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 1 to 12000
Data columns (total 11 columns):
last_session_creation_time    8823 non-null float64
opted_in_to_mailing_list      12000 non-null int64
enabled_for_marketing_drip    12000 non-null int64
org_id                        12000 non-null int64
invited_by_user_id            6417 non-null float64
GUEST_INVITE                  12000 non-null int64
ORG_INVITE                    12000 non-null int64
PERSONAL_PROJECTS             12000 non-null int64
SIGNUP                        12000 non-null int64
SIGNUP_GOOGLE_AUTH            12000 non-null int64
adopted_user                  12000 non-null bool
dtypes: bool(1), float64(2), int64(8)
memory usage: 1.3 MB


In [0]:
target = target.fillna(0)

In [13]:
target[target['adopted_user'] == True]

,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,GUEST_INVITE,ORG_INVITE,PERSONAL_PROJECTS,SIGNUP,SIGNUP_GOOGLE_AUTH,adopted_user
object_id,,,,,,,,,,,
10,1.401833e+09,1,1,318,4143.0,0,1,0,0,0,True
42,1.401045e+09,1,0,235,0.0,0,0,0,1,0,True
43,1.366010e+09,0,0,63,149.0,1,0,0,0,0,True
53,1.367798e+09,0,0,37,3641.0,1,0,0,0,0,True
63,1.401899e+09,0,0,203,0.0,0,0,0,0,1,True
...,...,...,...,...,...,...,...,...,...,...,...
11965,1.400657e+09,0,0,65,11251.0,1,0,0,0,0,True
11967,1.401524e+09,0,0,15,5688.0,1,0,0,0,0,True
11969,1.401411e+09,1,1,52,6647.0,1,0,0,0,0,True


In [14]:
target

,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,GUEST_INVITE,ORG_INVITE,PERSONAL_PROJECTS,SIGNUP,SIGNUP_GOOGLE_AUTH,adopted_user
object_id,,,,,,,,,,,
1,1.398139e+09,1,0,11,10803.0,1,0,0,0,0,False
2,1.396238e+09,0,0,1,316.0,0,1,0,0,0,False
3,1.363735e+09,0,0,94,1525.0,0,1,0,0,0,False
4,1.369210e+09,0,0,1,5151.0,1,0,0,0,0,False
5,1.358850e+09,0,0,193,5240.0,1,0,0,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...
11996,1.378448e+09,0,0,89,8263.0,0,1,0,0,0,False
11997,1.358275e+09,0,0,200,0.0,0,0,0,0,1,False
11998,1.398603e+09,1,1,83,8074.0,1,0,0,0,0,False


From the above analysis, we can see that of the 12000 users that signed up, there are 1445 that are "adopted users".

In [0]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [0]:
X = target.iloc[:, 0:10]
y = target.iloc[:, -1]

In [0]:
bestfeatures = SelectKBest(score_func=chi2, k=10)
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score'] 

In [18]:
print(featureScores.nlargest(10,'Score'))

                        Specs         Score
0  last_session_creation_time  2.374795e+11
4          invited_by_user_id  2.440980e+04
3                      org_id  5.160747e+03
7           PERSONAL_PROJECTS  5.530374e+01
5                GUEST_INVITE  2.110735e+01
9          SIGNUP_GOOGLE_AUTH  1.048737e+01
8                      SIGNUP  2.128383e+00
6                  ORG_INVITE  8.226464e-01
1    opted_in_to_mailing_list  2.263642e-01
2  enabled_for_marketing_drip  5.440139e-02


Factors that can help contribute in predicting future user adoption are 'last_session_creation_time', 'invited_by_user_id', 'org_id', and 'creation_source'.